In [26]:
pow(3, 12) * 12

6377292

In [36]:
from typing import List
from functools import lru_cache

# 二分搜索 + 状态压缩
class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        # 每一个工作都对一个工人来讲，都可以做，或者不做
        @lru_cache(None)
        def dfs(state, th, m): # 给第m个人，分配工作，并且这个工作时间不能超过th
            if state == 0:
                return True
            if m == k:
                return False
            subset = state
            while subset:
                if times[subset] <= th and dfs(state - subset, th, m+1):
                    return True
                subset = (subset - 1) & state
            return False
            
        n = len(jobs)
        dp = [[float('inf')] * (1 << n) for _ in range(k + 1)]
        times = [0] * (1 << n)
        for state in range(1 << n):
            sum_time = 0
            for i in range(n):
                if ((state >> i) & 1) == 1:
                    sum_time += jobs[i]
            times[state] = sum_time
        
        left, right = 1, sum(jobs)
        while left < right:
            mid = left + (right - left) // 2
            if dfs((1<<n)-1, mid, 0):
                right = mid
            else:
                left = mid + 1
        return left

In [48]:
from typing import List
from functools import lru_cache

# 二分搜索 + DFS
class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        
        def dfs(idx, workers, threshold):
            # idx：分配第 idx 件jobs，threshold：每个工人工作时间不能超过threshold
            # workers：每个工人工作的时间是多少
            if idx == n: # 完成了所有的工作
                return True
            flag = 0
            for i, w in enumerate(workers):
                if w + jobs[idx] > threshold:
                    continue
                if w == 0 and flag == 1:
                    continue
                if w == 0 and flag == 0:
                    flag = 1
                workers[i] += jobs[idx]
                if dfs(idx+1, workers, threshold):
                    return True
                workers[i] -= jobs[idx]
            return False
        
        low = 1
        high = sum(jobs)
        n = len(jobs)
        jobs.sort(reverse=True)
        while low < high:
            workers = [0] * k
            mid = low + (high - low) // 2
            if dfs(0, workers, mid):
                high = mid
            else:
                low = mid + 1


In [49]:
solution = Solution()
solution.minimumTimeRequired(jobs = [3,2,3], k = 3)

3

In [ ]:
from typing import List
from functools import lru_cache

# DFS + 二分搜索 + 状态压缩 TLE
class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        
        def dfs(workers, th, idx):
            if idx == len(jobs): # 所有的任务都分配完成
                return True
            faliure = {}
            for j in range(k):
                if workers[j] + jobs[idx] >  th:
                    continue
                if workers[j] in faliure and jobs[idx] >= faliure[workers[j]]:
                    continue
                
                workers[j] += jobs[idx]
                if dfs(workers, th, idx + 1):
                    return True
                workers[j] -= jobs[idx]
                if workers[j] in faliure:
                    faliure[workers[j]] = min(jobs[idx], faliure[workers[j]])
                else:
                    faliure[workers[j]] = jobs[idx]
            return False
        
        jobs.sort(reverse=True)
        left, right = 1, sum(jobs)
        while left < right:
            workers = [0] * k # 表示每个人分配的工作时长有多少
            mid = left + (right - left) // 2
            if dfs(workers, mid, 0): # 其中 0 表示分配第 idx=0 的工作给工人
                right = mid
            else:
                left = mid + 1
        return left

In [4]:
from typing import List

class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        @lru_cache(None)
        def calcTime(state):
            cost = 0
            for i in range(n):
                if ((state >> i) & 1) == 1:
                    cost += jobs[i]
            return cost
        
        n = len(jobs)
        m = 1 << n # 给一个人可能配工作的方式
        # dp[i][state]: 使用前 i 个工人，完成 state 工作，需要最少的工作时间是多少
        dp = [[float('inf')] * m for _ in range(k)]
        dp[0][0] = 0 # 没有一个人，完成 0 项工作，时间为 0
        
        for state in range(m):
            dp[0][state] = calcTime(state)
        
        for i in range(1, k):
            for state in range(m):
                subset = state
                while subset:
                    # 前 dp[i-1]个工人完成工作的最大时间
                    dp[i][state] = min(dp[i][state], max(dp[i-1][state-subset], calcTime(subset)))
                    subset = ((subset - 1) & state)
        return dp[-1][-1]

In [5]:
solution = Solution()
solution.minimumTimeRequired(jobs = [3,2,3], k = 3)

3